In [15]:
import pandas as pd

df = pd.read_csv('cohort_coad_read.2024-12-05.tsv', sep='\t')

In [16]:
df_sorted = df.sort_values(by='Sample Type')
df_sorted = df_sorted[["Sample Type", "File Name"]]
df_normal = df_sorted[df_sorted["Sample Type"] == "Solid Tissue Normal"]
df_tumor = df_sorted[df_sorted["Sample Type"] == "Primary Tumor"]
df_normal.sort_values(by='File Name', inplace=True)
df_tumor.sort_values(by='File Name', inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_8180\1357659582.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_normal.sort_values(by='File Name', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_8180\1357659582.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tumor.sort_values(by='File Name', inplace=True)


In [17]:
def split_string_by_underline(string):
    return string.split('_')

In [18]:
df_concat = pd.concat([df_normal, df_tumor], axis=0)
df_concat.reset_index(drop=True, inplace=True)
df_concat["Sample_Name"] = df_concat.index + 1
df_concat["Sample_Plate"] = ""
df_concat["Pool_ID"] = ""
# rename Sample Type to Sample_Group
df_concat.rename(columns={"Sample Type": "Sample_Group"}, inplace=True)
# Sentrix_ID is the fist element of the split string
# Sentrix_Position is the second element of the split string

df_concat["Sentrix_ID"] = df_concat["File Name"].apply(lambda x: split_string_by_underline(x)[0])
df_concat["Sentrix_Position"] = df_concat["File Name"].apply(lambda x: split_string_by_underline(x)[1])
df_concat["Sample_Well"] = ""
df_concat.drop(columns=["File Name"], inplace=True)

In [19]:
df_concat = df_concat[["Sample_Name", "Sample_Plate", "Sample_Group", "Pool_ID", "Sentrix_ID", "Sentrix_Position", "Sample_Well"]]
df_concat['Sample_Group'] = df_concat['Sample_Group'].str.replace(' ', '_')
df_concat

,Sample_Name,Sample_Plate,Sample_Group,Pool_ID,Sentrix_ID,Sentrix_Position,Sample_Well
0,1,,Solid_Tissue_Normal,,08b93e42-3a3a-4114-9185-ddaf728cc28f,noid,
1,2,,Solid_Tissue_Normal,,08b93e42-3a3a-4114-9185-ddaf728cc28f,noid,
2,3,,Solid_Tissue_Normal,,09cf025a-359f-4e4f-a6e2-05c30870e7d5,noid,
3,4,,Solid_Tissue_Normal,,09cf025a-359f-4e4f-a6e2-05c30870e7d5,noid,
4,5,,Solid_Tissue_Normal,,12969df6-305c-45a4-aaef-5587a9f9153c,noid,
...,...,...,...,...,...,...,...
905,906,,Primary_Tumor,,ff0532d5-a7ee-4236-bad1-bcfbf576a570,noid,
906,907,,Primary_Tumor,,ff1b9963-1886-4373-875d-712623d56697,noid,
907,908,,Primary_Tumor,,ff1b9963-1886-4373-875d-712623d56697,noid,
908,909,,Primary_Tumor,,ffc27d5a-29c6-44f9-bcb6-297a30ab24f5,noid,


In [20]:
df_concat.to_csv('gdc_sample_sheet.2024-12-05.modified.csv', index=False)

In [ ]:
import os
import shutil

def move_idat_files_to_parent(parent_dir):
    # Ensure the provided path is a directory
    if not os.path.isdir(parent_dir):
        print(f"{parent_dir} is not a valid directory.")
        return

    # Traverse through each subdirectory in the parent directory
    for subdir in os.listdir(parent_dir):
        subdir_path = os.path.join(parent_dir, subdir)

        # Check if it is a directory
        if os.path.isdir(subdir_path):
            # Iterate through files in the subdirectory
            for file in os.listdir(subdir_path):
                if file.endswith(".idat"):
                    file_path = os.path.join(subdir_path, file)
                    dest_path = os.path.join(parent_dir, file)

                    # Move the .idat file to the parent directory
                    try:
                        shutil.move(file_path, dest_path)
                        print(f"Moved: {file_path} -> {dest_path}")
                    except Exception as e:
                        print(f"Error moving {file_path}: {e}")

if __name__ == "__main__":
    parent_directory = "."  # Replace with the path to your parent directory
    move_idat_files_to_parent(parent_directory)
